# Correlation analysis for the apartments with themselves

<div class="alert alert-success" role="alert">
    <strong>Summary:</strong><br> This notebook calculates the correlation between apartment to form groups with similar consumption patterns. The first section of the notebook determines the correlation between apartments with themselves with a sampling of 10-min then we increase the timestep until the correlation hits a correlation above 0.5. At the same time, we explore the second hypothesis which is that the correlation is higher for the mean value of the three apartments. As before we increase the timestep as long as the threshold is not respected.
</div>

Import the needed modules in the notebook.

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from statsmodels.datasets import get_rdataset

Get the list of the files containing the Domestic Hot Water (DHW) consumption data using the key work 'IECS'.

In [2]:
folder = r"./Data/"
files = os.listdir(folder)
#Get a list of the different files named IECS
list_IECS = [file for file in files if '-IECS' in file]   
list_IECS.sort()

Read the csv file and resample the data for 10 min.

In [3]:
data = {} #Creation of a dictionary
for file in list_IECS:
    df = pd.read_csv(folder + file) #Read the csv file
    ts = df.set_index('0')['Value']     # DataFrame -> TimeSeries
    ts.index = pd.to_datetime(ts.index, unit='s')   # index to secondes
    ts = ts.resample("10Min").mean()    # resample 10 min
    data[file[:-4]] = ts 

Create dataframe from the data got previously.

In [4]:
df = pd.DataFrame(data)
df = df[~df.isnull().any(axis=1)]  # remove the row with Nan Value

Drop the columns that we don't need for our analysis and add the mean column of the three remaining apartments

In [5]:
df=df.drop(columns=['25-IECS', '26-IECS', '27-IECS', '35-IECS', '64-IECS', '65-IECS', '67-IECS', '9-IECS' ])
df['mean'] = df.mean(axis=1) 
#print(df)

Conversion of the water consumption from l/s to l/h.

In [17]:
df=df*3600
print(df)

                     34-IECS   36-IECS  63-IECS     mean
0                                                       
2020-12-01 12:10:00      0.0       0.0      0.0      0.0
2020-12-01 12:20:00      0.0       0.0      0.0      0.0
2020-12-01 12:30:00      0.0       0.0      0.0      0.0
2020-12-01 12:40:00  21384.0       0.0      0.0   7128.0
2020-12-01 12:50:00      0.0       0.0  20736.0   6912.0
...                      ...       ...      ...      ...
2021-06-08 14:50:00      0.0  213840.0      0.0  71280.0
2021-06-08 15:00:00      0.0   42768.0      0.0  14256.0
2021-06-08 15:10:00      0.0       0.0      0.0      0.0
2021-06-08 15:20:00      0.0   63504.0      0.0  21168.0
2021-06-08 15:30:00      0.0       0.0      0.0      0.0

[26089 rows x 4 columns]


## Correlation of the 3 apartments with themselves and of the mean value of the apartment starting with a comparison between December 2020 and January 2021

<div class="alert alert-info">
<strong>Details :</strong><br>
Working out the correlation of the apartments with themselves and for their mean value. If not sufficient, increase the resampling timestep. For this analysis we need to divide our information into two dataframe to compare them together.
</div>

We start by dividing our data into two dataframe, the first analysis is composed of the month of December 2020 and January 2021, we will compare their values to see if ten-minute sampling of december is equivalent to ten-minute sampling of january. It is essential to have the same number of columns and rows for this analysis either way the correlation won't work. To do so, we analyse our values by printing the data and looking at the numbers of rows and columns. We must change the dataset until the number of rows is equal to the other one.

In [7]:
df1=df['2020-12-01 00:00:00' : '2020-12-31 00:00:00']
df2=df['2021-01-01 00:00:00' : '2021-01-30 13:00:00']
print(df1)
print(df2)

                     34-IECS  36-IECS  63-IECS  mean
0                                                   
2020-12-01 12:10:00     0.00      0.0     0.00  0.00
2020-12-01 12:20:00     0.00      0.0     0.00  0.00
2020-12-01 12:30:00     0.00      0.0     0.00  0.00
2020-12-01 12:40:00     5.94      0.0     0.00  1.98
2020-12-01 12:50:00     0.00      0.0     5.76  1.92
...                      ...      ...      ...   ...
2020-12-30 23:20:00     0.00      0.0     0.00  0.00
2020-12-30 23:30:00     0.00      0.0     0.00  0.00
2020-12-30 23:40:00     0.00      0.0     0.00  0.00
2020-12-30 23:50:00     0.00      0.0     0.00  0.00
2020-12-31 00:00:00     0.00      0.0     0.00  0.00

[4053 rows x 4 columns]
                     34-IECS  36-IECS  63-IECS   mean
0                                                    
2021-01-01 00:00:00      0.0      0.0     0.00   0.00
2021-01-01 00:10:00      0.0      0.0     0.00   0.00
2021-01-01 00:20:00      0.0      0.0     0.00   0.00
2021-01-01 00:30

When it is done and we have the same number of rows, we can apply the correlation to the dataset. To work out the correlation between two dataframe we use the function corrwith() that calculate the correlation between two dataframes.
However, to compare the dataframe it is necessary to have the same name of the different rows, either way the correlation will return Nan value. To do so, we use the function set_axis of the library pandas that sets the rows name of the dataframe 2 as the rows name of the dataframe 1.

In [8]:
df1h_corr =df1.corrwith(df2.set_axis(df1.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df1h_corr)

34-IECS   -0.017422
36-IECS   -0.021889
63-IECS   -0.025402
mean      -0.058032
dtype: float64


As the value of the correlation are insufficient, we increase the timeset of the sampling to one hour.

In [9]:
df3=df1.resample('H').mean()
df4=df2.resample('H').mean().drop('2021-01-30 13:00:00')

print(df3)
print(df4)

df2h_corr =df3.corrwith(df4.set_axis(df3.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df2h_corr)

                     34-IECS  36-IECS  63-IECS    mean
0                                                     
2020-12-01 12:00:00    1.188     0.00    1.152   0.780
2020-12-01 13:00:00    4.950     0.96    5.880   3.930
2020-12-01 14:00:00   10.890     0.00   17.760   9.550
2020-12-01 15:00:00    0.000    14.76   15.780  10.180
2020-12-01 16:00:00    0.000     0.00    0.000   0.000
...                      ...      ...      ...     ...
2020-12-30 20:00:00    0.000     0.00    2.376   0.792
2020-12-30 21:00:00    1.980     0.96    0.000   0.980
2020-12-30 22:00:00    0.000     0.96    0.000   0.320
2020-12-30 23:00:00    0.000     0.00    0.000   0.000
2020-12-31 00:00:00    0.000     0.00    0.000   0.000

[709 rows x 4 columns]
                     34-IECS  36-IECS  63-IECS    mean
0                                                     
2021-01-01 00:00:00    0.000    0.000     0.00   0.000
2021-01-01 01:00:00    0.000    0.960     0.00   0.320
2021-01-01 02:00:00    0.000    0.000    

In [10]:
df5=df1.resample('10D').mean().drop('2020-12-31')
df6=df2.resample('10D').mean()

print(df5)
print(df6)

df3h_corr =df5.corrwith(df6.set_axis(df5.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df3h_corr)

             34-IECS   36-IECS   63-IECS      mean
0                                                 
2020-12-01  3.701524  3.629634  7.316067  4.882409
2020-12-11  4.396149  4.105996  5.959037  4.820394
2020-12-21  4.758890  2.833192  4.808327  4.133470
             34-IECS   36-IECS   63-IECS      mean
0                                                 
2021-01-01  4.944077  4.309442  2.202747  3.818755
2021-01-11  4.208430  3.687904  7.284388  5.060240
2021-01-21  4.650770  3.005076  6.574079  4.743308
34-IECS   -0.553395
36-IECS    0.640311
63-IECS   -0.822186
mean      -0.342955
dtype: float64


## Comparing the month of December

We want to compare the data of the month of December so we divide the month into two equal dataset.

In [11]:
df7=df['2020-12-01 00:00:00' : '2020-12-16 00:00:00']
df8=df['2020-12-16 00:10:00' : '2020-12-30 16:00:00']
print(df7)
print(df8)


df4h_corr =df7.corrwith(df8.set_axis(df7.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df4h_corr)

                     34-IECS  36-IECS  63-IECS  mean
0                                                   
2020-12-01 12:10:00     0.00      0.0     0.00  0.00
2020-12-01 12:20:00     0.00      0.0     0.00  0.00
2020-12-01 12:30:00     0.00      0.0     0.00  0.00
2020-12-01 12:40:00     5.94      0.0     0.00  1.98
2020-12-01 12:50:00     0.00      0.0     5.76  1.92
...                      ...      ...      ...   ...
2020-12-15 23:10:00     5.94      0.0     0.00  1.98
2020-12-15 23:30:00    17.82      0.0     0.00  5.94
2020-12-15 23:40:00     0.00      0.0     0.00  0.00
2020-12-15 23:50:00    11.88      0.0     0.00  3.96
2020-12-16 00:00:00     0.00      0.0     0.00  0.00

[2004 rows x 4 columns]
                     34-IECS  36-IECS  63-IECS  mean
0                                                   
2020-12-16 00:10:00      0.0      0.0      0.0   0.0
2020-12-16 00:20:00      0.0      0.0      0.0   0.0
2020-12-16 00:30:00      0.0      0.0      0.0   0.0
2020-12-16 00:40:00  

As the value of the correlation are insufficient, we increase the timeset of the sampling to one hour and drop the needed number of value to get the same number of rows.

In [12]:
df9=df7.resample('H').mean()
df10=df8.resample('H').mean()

print(df9)
print(df10)

df10=df10.drop('2020-12-30 13:00:00')
df10=df10.drop('2020-12-30 14:00:00')
df10=df10.drop('2020-12-30 15:00:00')
df10=df10.drop('2020-12-30 16:00:00')

print(df10)

df5h_corr =df9.corrwith(df10.set_axis(df9.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df5h_corr)

                     34-IECS  36-IECS  63-IECS    mean
0                                                     
2020-12-01 12:00:00    1.188     0.00    1.152   0.780
2020-12-01 13:00:00    4.950     0.96    5.880   3.930
2020-12-01 14:00:00   10.890     0.00   17.760   9.550
2020-12-01 15:00:00    0.000    14.76   15.780  10.180
2020-12-01 16:00:00    0.000     0.00    0.000   0.000
...                      ...      ...      ...     ...
2020-12-15 20:00:00    5.940     0.96    7.860   4.920
2020-12-15 21:00:00    7.920     0.00    8.820   5.580
2020-12-15 22:00:00    8.910     0.00   26.760  11.890
2020-12-15 23:00:00    7.128     0.00    0.000   2.376
2020-12-16 00:00:00    0.000     0.00    0.000   0.000

[349 rows x 4 columns]
                     34-IECS  36-IECS  63-IECS  mean
0                                                   
2020-12-16 00:00:00     0.00     0.00     0.00  0.00
2020-12-16 01:00:00     0.00     0.00     0.00  0.00
2020-12-16 02:00:00     0.00     0.00     0.00  0

The threshold is still unrespected so we increase the timeset

In [14]:
df11=df7.resample('D').mean()
df12=df8.resample('D').mean()

print(df11)
print(df12)

df11=df11.drop('2020-12-16')

df6h_corr =df11.corrwith(df12.set_axis(df11.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df6h_corr)

             34-IECS   36-IECS    63-IECS      mean
0                                                  
2020-12-01  3.936176  4.500000   7.740000  5.392059
2020-12-02  1.740857  3.648857   4.047429  3.145714
2020-12-03  3.040292  2.267737   5.888759  3.732263
2020-12-04  2.320147  2.459118   7.197353  3.992206
2020-12-05  4.546619  5.630504   6.265036  5.480719
2020-12-06  5.186331  5.876547   8.476835  6.513237
2020-12-07  4.612993  3.804964   5.352701  4.590219
2020-12-08  1.994453  2.435912   6.070073  3.500146
2020-12-09  4.602302  3.426475  11.305036  6.444604
2020-12-10  5.108143  2.638286  10.962000  6.236143
2020-12-11  4.787737  1.968175   9.741022  5.498978
2020-12-12  4.884604  2.566619   9.152806  5.534676
2020-12-13  4.048029  9.417810   7.173723  6.879854
2020-12-14  2.867050  2.152230   5.829928  3.616403
2020-12-15  3.078129  3.286619   6.417842  4.260863
2020-12-16  0.000000  0.000000   0.000000  0.000000
             34-IECS   36-IECS   63-IECS      mean
0            

The threshold was respected for only one apartment so we increase the timeset to explore the differences

In [16]:
df13=df7.resample('5D').mean()
df14=df8.resample('5D').mean()

print(df13)
print(df14)

df13=df13.drop('2020-12-16')

df7h_corr =df13.corrwith(df14.set_axis(df13.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df7h_corr)

             34-IECS   36-IECS   63-IECS      mean
0                                                 
2020-12-01  3.024871  3.620323  6.047419  4.230871
2020-12-06  4.307775  3.637977  8.452717  5.466156
2020-12-11  3.930304  3.867786  7.658466  5.152185
2020-12-16  0.000000  0.000000  0.000000  0.000000
             34-IECS   36-IECS   63-IECS      mean
0                                                 
2020-12-16  4.876701  4.354462  4.238351  4.489838
2020-12-21  4.439464  2.517321  5.789464  4.248750
2020-12-26  4.896662  2.836447  3.832098  3.855069
34-IECS   -0.699922
36-IECS   -0.411685
63-IECS    0.609686
mean      -0.614913
dtype: float64


## Comparing two dataframe of six months

We repeat our experimentation with two dataframe with six months of data.

In [31]:
df15=df['2020-12-01 00:00:00' : '2021-02-28 00:00:00']
df16=df['2021-03-01 00:00:00' : '2021-05-28 04:00:00']
print(df15)
print(df16)


df8h_corr =df15.corrwith(df16.set_axis(df15.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df8h_corr)

                     34-IECS  36-IECS  63-IECS    mean
0                                                     
2020-12-01 12:10:00      0.0      0.0      0.0     0.0
2020-12-01 12:20:00      0.0      0.0      0.0     0.0
2020-12-01 12:30:00      0.0      0.0      0.0     0.0
2020-12-01 12:40:00  21384.0      0.0      0.0  7128.0
2020-12-01 12:50:00      0.0      0.0  20736.0  6912.0
...                      ...      ...      ...     ...
2021-02-27 23:20:00      0.0      0.0      0.0     0.0
2021-02-27 23:30:00      0.0      0.0      0.0     0.0
2021-02-27 23:40:00      0.0      0.0      0.0     0.0
2021-02-27 23:50:00      0.0      0.0      0.0     0.0
2021-02-28 00:00:00      0.0      0.0      0.0     0.0

[12187 rows x 4 columns]
                     34-IECS  36-IECS  63-IECS  mean
0                                                   
2021-03-01 00:00:00      0.0      0.0      0.0   0.0
2021-03-01 00:10:00      0.0      0.0      0.0   0.0
2021-03-01 00:20:00      0.0      0.0      0.0 

The values of the correlation are not sufficient so we increase the timestep to one-hour resampling.

In [35]:
df17=df15.resample('H').mean()
df18=df16.resample('H').mean()
print(df17)
print(df18)

df17=df17.drop('2020-12-01 12:00:00')
df17=df17.drop('2020-12-01 13:00:00')
df17=df17.drop('2020-12-01 14:00:00')
df17=df17.drop('2020-12-01 15:00:00')
df17=df17.drop('2020-12-01 16:00:00')
df17=df17.drop('2020-12-01 17:00:00')
df17=df17.drop('2020-12-01 18:00:00')
df17=df17.drop('2020-12-01 19:00:00')

print(df17.shape)

df9h_corr =df17.corrwith(df18.set_axis(df17.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df9h_corr)

                     34-IECS   36-IECS  63-IECS      mean
0                                                        
2020-12-01 12:00:00   4276.8       0.0   4147.2    2808.0
2020-12-01 13:00:00  17820.0    3456.0  21168.0   14148.0
2020-12-01 14:00:00  39204.0       0.0  63936.0   34380.0
2020-12-01 15:00:00      0.0   53136.0  56808.0   36648.0
2020-12-01 16:00:00      0.0       0.0      0.0       0.0
...                      ...       ...      ...       ...
2021-02-27 20:00:00  81777.6  298339.2  46915.2  142344.0
2021-02-27 21:00:00      0.0    3456.0  96336.0   33264.0
2021-02-27 22:00:00      0.0       0.0      0.0       0.0
2021-02-27 23:00:00      0.0       0.0      0.0       0.0
2021-02-28 00:00:00      0.0       0.0      0.0       0.0

[2125 rows x 4 columns]
                     34-IECS  36-IECS  63-IECS     mean
0                                                      
2021-03-01 00:00:00      0.0      0.0      0.0      0.0
2021-03-01 01:00:00      0.0      0.0  64152.0  21384

Increasing the timestep to a day resampling

In [38]:
df19=df15.resample('D').mean()
df20=df16.resample('D').mean()
print(df19)
print(df20)

df19=df19.drop('2020-12-01')

print(df19.shape)

df10h_corr =df19.corrwith(df20.set_axis(df19.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df10h_corr)

                 34-IECS       36-IECS       63-IECS          mean
0                                                                 
2020-12-01  14170.235294  16200.000000  27864.000000  19411.411765
2020-12-02   6267.085714  13135.885714  14570.742857  11324.571429
2020-12-03  10945.051095   8163.854015  21199.532847  13436.145985
2020-12-04   8352.529412   8852.823529  25910.470588  14371.941176
2020-12-05  16367.827338  20269.812950  22554.129496  19730.589928
...                  ...           ...           ...           ...
2021-02-24  13388.823529  11826.000000  20993.294118  15402.705882
2021-02-25   4773.755396  13761.841727  22675.338129  13736.978417
2021-02-26  12700.800000  12979.200000  38227.200000  21302.400000
2021-02-27  16758.131387  40166.540146  54602.277372  37175.649635
2021-02-28      0.000000      0.000000      0.000000      0.000000

[90 rows x 4 columns]
                 34-IECS       36-IECS       63-IECS          mean
0                                      

Not enough so increase the timestep to 10-days

In [40]:
df21=df15.resample('10D').mean()
df22=df16.resample('10D').mean()
print(df19)
print(df20)



df11h_corr =df21.corrwith(df22.set_axis(df21.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df11h_corr)

                 34-IECS       36-IECS       63-IECS          mean
0                                                                 
2020-12-02   6267.085714  13135.885714  14570.742857  11324.571429
2020-12-03  10945.051095   8163.854015  21199.532847  13436.145985
2020-12-04   8352.529412   8852.823529  25910.470588  14371.941176
2020-12-05  16367.827338  20269.812950  22554.129496  19730.589928
2020-12-06  18670.791367  21155.568345  30516.604317  23447.654676
...                  ...           ...           ...           ...
2021-02-24  13388.823529  11826.000000  20993.294118  15402.705882
2021-02-25   4773.755396  13761.841727  22675.338129  13736.978417
2021-02-26  12700.800000  12979.200000  38227.200000  21302.400000
2021-02-27  16758.131387  40166.540146  54602.277372  37175.649635
2021-02-28      0.000000      0.000000      0.000000      0.000000

[89 rows x 4 columns]
                 34-IECS       36-IECS       63-IECS          mean
0                                      

Not enough so increase to one month.

In [41]:
df23=df15.resample('M').mean()
df24=df16.resample('M').mean()
print(df23)
print(df24)



df12h_corr =df23.corrwith(df24.set_axis(df23.index, axis='index', copy=False)) # Correlation matrice of df1h with df2 changing the index name with the index name of dh1 to compare them
print(df12h_corr)

                 34-IECS       36-IECS       63-IECS          mean
0                                                                 
2020-12-31  15520.473170  12573.950870  21168.412115  16420.945385
2021-01-31  17275.077068  13580.289474  20182.398496  17012.588346
2021-02-28  13504.160514  13503.293740  25771.955056  17593.136437
                 34-IECS       36-IECS       63-IECS          mean
0                                                                 
2021-03-31  16592.846226  14369.233924  22736.465983  17899.515377
2021-04-30  14288.572257  14140.117931  18566.499758  15665.063316
2021-05-31  11751.791323  14700.648390  10460.448230  12304.295981
34-IECS    0.557498
36-IECS    0.036186
63-IECS   -0.874419
mean      -0.992668
dtype: float64


Enough for one apartment but not for the other and clearly not the mean value.